In [1]:
using Revise

INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc-2/packages/lib/v0.6/Revise.ji for module Revise.


In [5]:
using RigidBodyDynamics
using AtlasRobot

INFO: Precompiling module AtlasRobot.


In [6]:
robot = AtlasRobot.mechanism()

Spanning tree:
Vertex: world (root)
  Vertex: pelvis, Edge: pelvis_to_world
    Vertex: ltorso, Edge: back_bkz
      Vertex: mtorso, Edge: back_bky
        Vertex: utorso, Edge: back_bkx
          Vertex: l_clav, Edge: l_arm_shz
            Vertex: l_scap, Edge: l_arm_shx
              Vertex: l_uarm, Edge: l_arm_ely
                Vertex: l_larm, Edge: l_arm_elx
                  Vertex: l_ufarm, Edge: l_arm_uwy
                    Vertex: l_lfarm, Edge: l_arm_mwx
                      Vertex: l_hand, Edge: l_arm_lwy
          Vertex: head, Edge: neck_ay
          Vertex: r_clav, Edge: r_arm_shz
            Vertex: r_scap, Edge: r_arm_shx
              Vertex: r_uarm, Edge: r_arm_ely
                Vertex: r_larm, Edge: r_arm_elx
                  Vertex: r_ufarm, Edge: r_arm_uwy
                    Vertex: r_lfarm, Edge: r_arm_mwx
                      Vertex: r_hand, Edge: r_arm_lwy
    Vertex: l_uglut, Edge: l_leg_hpz
      Vertex: l_lglut, Edge: l_leg_hpx
        Vertex: l_uleg,

In [7]:
cache = StateCache(robot)

StateCache{…}(…)

In [8]:
using ForwardDiff

INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc-2/packages/lib/v0.6/ForwardDiff.ji for module ForwardDiff.


In [9]:
translation(transform_to_root(cache[Float64], default_frame(findbody(robot, "l_foot"))))

3-element StaticArrays.SArray{Tuple{3},Float64,1,3}:
  0.0   
  0.1115
 -0.862 

In [27]:
function tform(frame, q)
    state = cache[eltype(q)]
    set_configuration!(state, q)
    translation(transform_to_root(state, frame))
end

tform (generic function with 2 methods)

In [30]:
frame = default_frame(findbody(robot, "l_foot"))
q = configuration(MechanismState(robot))
ForwardDiff.jacobian(q -> tform(frame, q), q)

3×37 Array{Float64,2}:
 0.0  0.0    -1.724  -0.223  1.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.724   0.0     0.0    0.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.223   0.0     0.0    0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [31]:
using BenchmarkTools

INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc-2/packages/lib/v0.6/BenchmarkTools.ji for module BenchmarkTools.


In [32]:
@btime ForwardDiff.jacobian(q -> tform($frame, q), $q)

  212.050 μs (168 allocations: 55.44 KiB)


3×37 Array{Float64,2}:
 0.0  0.0    -1.724  -0.223  1.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.724   0.0     0.0    0.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.223   0.0     0.0    0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0